In [121]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [122]:
# Load data into a DataFrame
df=pd.read_excel("D:\Datasets\Dataset.xlsx");

In [123]:
df

,Article
0,"Retailers, the makers of foods marketed for we..."
1,"Move over, Ozempic — there’s a new drug in tow..."
2,Sept 14 (Reuters) - Bristol Myers Squibb (BMY....
3,Austin Wolcott was 18 years old and pretty sur...
4,"Cancer, often referred to as the “emperor of a..."
5,Nov 28 (Reuters) - The U.S. Food and Drug Admi...
6,Nov 21 (Reuters) - BeiGene (6160.HK) said on T...
7,Sept 19 (Reuters) - Drugmaker BeiGene (6160.HK...
8,BRUKINSA is the first and only BTK inhibitor a...
9,Whether you're looking for a quick bite to eat...


In [124]:
df.describe(include="O");

In [125]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Article  25 non-null     object
dtypes: object(1)
memory usage: 328.0+ bytes


In [126]:
#Check the null enteries
df.isnull().sum()

Article    0
dtype: int64

In [127]:
# df.fillna(df.mode());(if you got the null values but we don;t have any)
# # df.dropna();

In [128]:
# Display the first 10 rows to check the data
print(df.head())

                                             Article
0  Retailers, the makers of foods marketed for we...
1  Move over, Ozempic — there’s a new drug in tow...
2  Sept 14 (Reuters) - Bristol Myers Squibb (BMY....
3  Austin Wolcott was 18 years old and pretty sur...
4  Cancer, often referred to as the “emperor of a...


In [129]:
# Convert text to lowercase
df['Article'] = df['Article'].str.lower()

In [130]:
# Function to clean text
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Apply the cleaning function to the text
df['cleaned_text'] = df['Article'].apply(clean_text)

In [131]:
# Display the first few cleaned rows to check the data
print(df[['Article', 'cleaned_text']].head())

                                             Article  \
0  retailers, the makers of foods marketed for we...   
1  move over, ozempic — there’s a new drug in tow...   
2  sept 14 (reuters) - bristol myers squibb (bmy....   
3  austin wolcott was 18 years old and pretty sur...   
4  cancer, often referred to as the “emperor of a...   

                                        cleaned_text  
0  retailers makers foods marketed weight loss ty...  
1  move ozempic theres new drug town eli lillys z...  
2  sept 14 reuters bristol myers squibb bmyn said...  
3  austin wolcott 18 years old pretty sure wouldn...  
4  cancer often referred emperor maladies unyield...  


In [132]:
# Extract the texts
texts = df['cleaned_text'].tolist()

In [133]:
# Initialize the tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [134]:
# Convert text to sequences
sequences = tokenizer.texts_to_sequences(texts)

In [135]:
# Define maximum sequence length
maxlen = 256 

In [136]:
# Pad sequences
X=pad_sequences(sequences,maxlen=256,value=word_index['the'],padding='post')

In [137]:
# we have a binary classification (e.g., positive or negative sentiment)
# For demonstration, we'll create random labels (0: negative, 1: positive)
labels = np.random.randint(2, size=len(df))

In [138]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

In [139]:
X.shape

(25, 256)

In [140]:
type(labels)

numpy.ndarray

In [141]:
output_dim=128
max_words=256

In [142]:
# Build the LSTM model
model=Sequential()
model.add(Embedding(len(labels),output_dim,input_length=max_words))
model.add(LSTM(128,activation='tanh'))
model.add(Dense(1,activation='sigmoid'))# For binary classification

In [148]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
1/1 [==============================] - 6s 6s/step - loss: 0.5451 - acc: 0.7000 - val_loss: 0.7799 - val_acc: 0.4000
Epoch 2/10
1/1 [==============================] - 0s 280ms/step - loss: 0.4734 - acc: 0.7000 - val_loss: 1.2432 - val_acc: 0.4000
Epoch 3/10
1/1 [==============================] - 0s 264ms/step - loss: 0.4638 - acc: 0.5500 - val_loss: 0.7265 - val_acc: 0.4000
Epoch 4/10
1/1 [==============================] - 0s 267ms/step - loss: 0.3745 - acc: 0.9500 - val_loss: 0.7068 - val_acc: 0.4000
Epoch 5/10
1/1 [==============================] - 0s 275ms/step - loss: 0.3906 - acc: 0.9500 - val_loss: 0.7003 - val_acc: 0.4000
Epoch 6/10
1/1 [==============================] - 0s 293ms/step - loss: 0.3902 - acc: 0.9500 - val_loss: 0.6962 - val_acc: 0.6000
Epoch 7/10
1/1 [==============================] - 0s 264ms/step - loss: 0.3814 - acc: 0.9500 - val_loss: 0.6926 - val_acc: 0.6000
Epoch 8/10
1/1 [==============================] - 0s 279ms/step - loss: 0.3665 - acc: 0.9500 

In [151]:
# Predict the model
y_pred = model.predict(X_test)

1/1 [==============================] - 0s 86ms/step


In [95]:
model.predict(X_test)

1/1 [==============================] - 1s 924ms/step


array([[0.5321177 ],
       [0.5094392 ],
       [0.5056964 ],
       [0.51058996],
       [0.5127509 ]], dtype=float32)

In [ ]:
# Description

1. Install and Import Required Libraries: Ensure you have all the necessary libraries installed.

2. Input Data: Place your dataset.

# Tools/Libraries Used

1. Pandas: For data manipulation and analysis.
2. Numpy: For numerical operations.
3. NLTK: For natural language processing tasks.
4. Keras: For building and training neural networks.
5. Scikit-learn: For machine learning utilities and evaluation.

# Steps Implemented

1. Data Preprocessing: Cleaned and tokenized the text data.
2. Model Building: Built and trained an LSTM model.
3. Evaluation: Evaluated the model.
    
Step 1: Data Preparation
    
Extract the data from excel file and Load Data: Fetch data from given excel file and load it into a Pandas DataFrame.
Data Cleaning: Preprocess the text data by removing punctuation, converting to lowercase, and removing stopwords.
Tokenization and Padding: Tokenize the text and pad sequences for the LSTM model.
    
Step 2: Model Building

Train-Test Split: Split the data into training and testing sets.
Build LSTM Model: Define and compile the LSTM model.
Train the Model: Train the LSTM model on the training data.
    
Step 3: Evaluation
Evaluate the Model: Evaluate the model on the test data.
Predict and Analyze: Make predictions.